In [6]:
import sys
# sys.path.append('/Users/bernardoloureiro/template-lib')

from utils.notebookhelpers.helpers import Helpers
from utils.dtos.templateOutputCollection import TemplateOutputCollection
from utils.dtos.variable import Metadata
from utils.dtos.templateOutput import TemplateOutput
from utils.dtos.templateOutput import OutputType
from utils.dtos.templateOutput import ChartType
import datetime
import logging
import pandas as pd
import numpy as np
import os
from dateutil import parser
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)
import math
import scipy as scipy
import matplotlib.pyplot as plt
import seaborn as sns
import copy
%matplotlib inline
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import ADASYN
from sklearn.ensemble import RandomForestClassifier

from collections import Counter
from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve, auc
# from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.calibration import CalibratedClassifierCV

In [7]:
inputDatasetParameter=Helpers.get_or_create_input_dataset(
    name="inputBeneDataset3",
    metadata=Metadata(input_name='inpatient data', is_required=True, tooltip='inpatient  Dataset for EDA'),
    local_context=locals()
)

inputDatasetParameter2=Helpers.get_or_create_input_dataset(
    name="inputBeneDataset4",
    metadata=Metadata(input_name='outpatient data', is_required=True, tooltip=' outpatient Dataset for EDA'),
    local_context=locals()
)

inputDatasetParameter3=Helpers.get_or_create_input_dataset(
    name="inputBeneDataset5",
    metadata=Metadata(input_name='target label data', is_required=True, tooltip=' Label Dataset'),
    local_context=locals()
)

inputDatasetParameter4=Helpers.get_or_create_input_dataset(
    name="inputBeneDataset6",
    metadata=Metadata(input_name='train_bene_df', is_required=True, tooltip='Benefeciary Dataset'),
    local_context=locals()
)


# Gender = Helpers.get_or_create_input_var(
#     name="Gender",
#     metadata=Metadata(input_name="Longitude", is_required=True, tooltip="Longitude", multiple=False, datatypes=["DOUBLE", "LONG"], options=['FIELDS', 'CONSTANT'], datasets=['GeoDataset']),
#     local_context=locals()
# )

EDA_Inpatient_Outpatient=Helpers.get_or_create_output_chart(
 name="EDA_Inpatient_Outpatient_Data",
    metadata=Metadata(input_name='EDA_Inpatient_Outpatient_Data', is_required=True, tooltip='Name of the chart to be created'),
    local_context=locals()
)

In [8]:
contextId = "EDA_Inpatient_Outpatient_Data"
context = Helpers.getOrCreateContext(contextId = contextId, localVars=locals())

In [9]:
entities=Helpers.getAllEntities(context)
IpDataset = inputDatasetParameter.value
OpDataset = inputDatasetParameter2.value
train_bene_df=inputDatasetParameter4.value
train_tgt_Dataset = inputDatasetParameter3.value


EDA_Inpatient_Outpatient=EDA_Inpatient_Outpatient.value

KeyError: 'entityPaths'

In [ ]:
df = Helpers.getEntityData(context,IpDataset)
df2 = Helpers.getEntityData(context,OpDataset)
df4 = Helpers.getEntityData(context,train_bene_df)
df3 = Helpers.getEntityData(context,train_tgt_Dataset)


In [ ]:
train_ip_df = df.copy()
train_op_df = df2.copy()
train_tgt_lbls_df = df3.copy()
train_bene_df = df4.copy()

In [2]:
pd.set_option('display.max_columns',30)
label_font_dict = {'family':'sans-serif','size':13.5,'color':'brown','style':'italic'}
title_font_dict = {'family':'sans-serif','size':16.5,'color':'Blue','style':'italic'}

# EDA Start`

In [ ]:
train_op_df["Admitted?"] = 0
train_op_df['ClaimStartDt'] = pd.to_datetime(train_op_df['ClaimStartDt'], format="%Y-%m-%d")
train_op_df['ClaimEndDt'] = pd.to_datetime(train_op_df['ClaimEndDt'], format="%Y-%m-%d")
train_op_df['Claim_Duration'] = (train_op_df['ClaimEndDt'] - train_op_df['ClaimStartDt']).dt.days
ccc = train_op_df['Claim_Duration']

In [ ]:
# Here, I'm displaying the number of only in-patients and out-patients
with plt.style.context('seaborn-poster'):
    plt.figure(figsize=(12,8))
    ccc.plot(kind='hist', colormap="viridis");
    # Providing the labels and title to the graph
    plt.xlabel("Claim Duration(in days)", fontdict=label_font_dict)
    plt.minorticks_on()
    plt.title("Distribution of Claim Duration Days\n", fontdict=title_font_dict)
    plt.plot();

In [ ]:
imgChart = Helpers.createTemplateOutputPlotlibChart(context, chartTitle='Distribution of Claim Duration Days\n',
                                                    plt=plt)

# EDA On all Data

In [ ]:
with plt.style.context('seaborn-poster'):
    fig = train_tgt_lbls_df["PotentialFraud"].value_counts().plot(kind='bar', color=['green','orange'])
    # Using the "patches" function we will get the location of the rectangle bars from the graph.
    ## Then by using those location(width & height) values we will add the annotations
    for p in fig.patches:
        width = p.get_width()
        height = p.get_height()
        x, y = p.get_xy()
        fig.annotate(f'{str(round((height*100)/train_tgt_lbls_df.shape[0],2))+"%"}', (x + width/2, y + height*1.015), ha='center', fontsize=13.5)
    # Providing the labels and title to the graph
    plt.xlabel("Provider Fraud or Not?", fontdict=label_font_dict)
    plt.ylabel("Number or % share of providers\n", fontdict=label_font_dict)
    plt.yticks(np.arange(0,5100,500))
    plt.grid(which='major', linestyle="--", color='lightgrey')
    plt.minorticks_on()
    plt.title("Distribution of Fraud & Non-fraud providers\n", fontdict=title_font_dict)
    plt.plot();

In [ ]:
imgChart2 = Helpers.createTemplateOutputPlotlibChart(context, chartTitle='Distribution of Fraud & Non-fraud providers\n',
                                                    plt=plt)

In [ ]:
train_ip_df["Admitted?"] = 1
common_cols = [col for col in train_ip_df.columns if col in train_op_df.columns]
train_ip_df['ClaimStartDt'] = pd.to_datetime(train_ip_df['ClaimStartDt'], format="%Y-%m-%d")
train_ip_df['ClaimEndDt'] = pd.to_datetime(train_ip_df['ClaimEndDt'], format="%Y-%m-%d")
train_op_df['ClaimStartDt'] = pd.to_datetime(train_op_df['ClaimStartDt'], format="%Y-%m-%d")
train_op_df['ClaimEndDt'] = pd.to_datetime(train_op_df['ClaimEndDt'], format="%Y-%m-%d")
train_ip_op_df = pd.merge(left=train_ip_df, right=train_op_df, left_on=common_cols, right_on=common_cols, how="outer")
train_ip_op_bene_df = pd.merge(left=train_ip_op_df, right=train_bene_df, left_on='BeneID', right_on='BeneID',how='inner')
train_iobp_df = pd.merge(left=train_ip_op_bene_df, right=train_tgt_lbls_df, left_on='Provider', right_on='Provider',how='inner')

In [ ]:
prvs_claims_df = pd.DataFrame(train_iobp_df.groupby(['Provider'])['ClaimID'].count()).reset_index()
prvs_claims_tgt_lbls_df = pd.merge(left=prvs_claims_df, right=train_tgt_lbls_df, on='Provider', how='inner')

In [ ]:
print(pd.DataFrame(train_iobp_df['PotentialFraud'].value_counts()), "\n")

with plt.style.context('seaborn-poster'):
    fig = train_iobp_df['PotentialFraud'].value_counts().plot(kind='bar', color=['green','orange'])
    # Using the "patches" function we will get the location of the rectangle bars from the graph.
    ## Then by using those location(width & height) values we will add the annotations
    for p in fig.patches:
        width = p.get_width()
        height = p.get_height()
        x, y = p.get_xy()
        fig.annotate(f'{str(round((height*100)/train_iobp_df.shape[0],2))+"%"}', (x + width/2, y + height*1.015), ha='center', fontsize=13.5)
    # Providing the labels and title to the graph
    plt.xlabel("Fraud or Not?", fontdict=label_font_dict)
    plt.ylabel("Number (or %) of claims\n", fontdict=label_font_dict)
    plt.grid(which='major', linestyle="--", color='lightgrey')
    plt.minorticks_on()
    plt.title("Distribution of Fraud & Non-fraud claims\n", fontdict=title_font_dict)
    plt.plot();

In [ ]:
imgChart3 = Helpers.createTemplateOutputPlotlibChart(context, chartTitle='Distribution of Fraud & Non-fraud claims\n',
                                                    plt=plt)

In [ ]:
train_iobp_df['DOB'] = pd.to_datetime(train_iobp_df['DOB'], format="%Y-%m-%d")
train_iobp_df['DOD'] = pd.to_datetime(train_iobp_df['DOD'], format="%Y-%m-%d")
train_iobp_df['Is_Alive?'] = train_iobp_df['DOD'].apply(lambda val: 'No' if val != val else 'Yes')
train_iobp_df['ClaimStartDt'] = pd.to_datetime(train_iobp_df['ClaimStartDt'], format="%Y-%m-%d")
train_iobp_df['ClaimEndDt'] = pd.to_datetime(train_iobp_df['ClaimEndDt'], format="%Y-%m-%d")

train_iobp_df['Claim_Duration'] = (train_iobp_df['ClaimEndDt'] - train_iobp_df['ClaimStartDt']).dt.days
# tmp = train_iobp_df['Claim_Duration'] 

In [ ]:
with plt.style.context('seaborn'):
    fig = sns.boxenplot(data=train_iobp_df, x='PotentialFraud',y='Claim_Duration', palette='dark')
    # Providing the labels and title to the graph
    plt.xlabel("Potentially Fraud?", fontdict=label_font_dict)
    plt.xticks(rotation=90, fontsize=12)
    plt.ylabel("Claim Duration (in days)\n", fontdict=label_font_dict)
    plt.minorticks_on()
    plt.grid(which='major', linestyle="--", color='lightgrey')
    plt.title("Claim Duration for Potentially Fraud & Non-Fraud Providers\n", fontdict=title_font_dict)
    plt.plot();

In [ ]:
imgChart4 = Helpers.createTemplateOutputPlotlibChart(context, chartTitle='Claim Duration for Potentially Fraud & Non-Fraud Providers\n',
                                                    plt=plt)

In [ ]:
                                     


outputCollection = Helpers.createOutputCollection(context)


outputCollection.addTemplateOutput(imgChart)
outputCollection.addTemplateOutput(imgChart2)
outputCollection.addTemplateOutput(imgChart3)
outputCollection.addTemplateOutput(imgChart4)
# outputCollection.addTemplateOutput(imgChart5)
# outputCollection.addTemplateOutput(imgChart6)
# outputCollection.addTemplateOutput(imgChart7)
# outputCollection.addTemplateOutput(imgChart8)
# outputCollection.addTemplateOutput(imgChart9)
# outputCollection.addTemplateOutput(imgChart10)

# outputCollection.addTemplateOutput(imgChart11)
# outputCollection.addTemplateOutput(imgChart12)
# outputCollection.addTemplateOutput(imgChart13)
# outputCollection.addTemplateOutput(imgChart14)
# # outputCollection.addTemplateOutput(imgChart15)
# outputCollection.addTemplateOutput(imgChart16)
# outputCollection.addTemplateOutput(imgChart17)
# outputCollection.addTemplateOutput(imgChart18)
# outputCollection.addTemplateOutput(imgChart19)
# outputCollection.addTemplateOutput(imgChart20)
# outputCollection.addTemplateOutput(imgChart21)



Helpers.save(context)

Helpers.save(context)